<span style="color:black;font-size:30pt"> Mitochondria assembly and species identification


<span style="color:blue;font-size:14pt"> We start by assembling the raw nanopore reads. We use genomeSize=16k as it is the expected size of a mitochondrial genome. </span>

In [34]:
 ~/canu/Linux-amd64/bin/canu -d assemble/ -p shark genomeSize=16k minReadLength=500 -nanopore-raw all_shark.fastq 2> canu.log


<span style="color:blue;font-size:14pt">we then blast the contigs agains nt

In [39]:
#cp assemble/shark.contigs.fasta shark.contigs.fasta
export BLASTDB=/home/ubuntu/DB
blastn -db nt -query shark.contigs.fasta -out contigs_nt.blastn -outfmt "6 qseqid sseqid pident length evalue salltitles sallacc sstart send qstart qend" -max_target_seqs 2  -max_hsps 2 -num_threads 16 -culling_limit 1 -evalue 0.001


In [40]:
cat contigs_nt.blastn

tig00000001	gi|19067985|gb|AY049832.1|	93.27	4025	0.0	Galeocerdo cuvier internal transcribed spacer 2, and 28S ribosomal RNA gene, partial sequence	AY049832	3987	9	258	4191
tig00000001	gi|19067986|gb|AY049833.1|	92.22	1787	0.0	Galeocerdo cuvier 18S ribosomal RNA gene, complete sequence	AY049833	1780	1	9695	11431
tig00000003	gi|156119673|gb|AC164927.3|	72.46	973	4e-68	Ginglymostoma cirratum clone GC_Ba-557B6, complete sequence	AC164927	38782	39736	307	1253
tig00000003	gi|301128892|emb|FQ032660.1|	96.15	52	2e-11	Scyliorhinus canicula Cluster_HOXD sequence	FQ032660	24936	24987	3	53
tig00000010	gi|456368036|gb|KC470543.1|	93.08	14116	0.0	Carcharhinus obscurus mitochondrion, complete genome	KC470543	1	14102	8310	22174
tig00000010	gi|456368036|gb|KC470543.1|	91.96	8474	0.0	Carcharhinus obscurus mitochondrion, complete genome	KC470543	8239	16706	1	8309
tig00000010	gi|1382859157|gb|MG912791.1|	73.40	7698	0.0	Chelonoidis duncanensis isolate 8291 mitochondrion, partial genome	MG912791	14022	6492

<span style="color:blue;font-size:14pt"> tig00000010 has a long hit to "Carcharhinus obscurus mitochondrion". let's extract it </span>

In [9]:
samtools faidx shark.contigs.fasta 'tig00000010' > tig00000010.fasta

<span style="color:blue;font-size:14pt"> And let's also download the Carcharhinus obscurus mitochondrion genome

In [21]:
curl 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&id=KC470543&retnode=text&rettype=fasta' > 'Carcharhinus_obscurus_mitochondrion.fasta' 2> /dev/null


<span style="color:blue;font-size:14pt">Using Gepard we construct a dotplot of tig00000010 against itself. Note that the start and end of the sequence is highly similar. This is common when you assemble a circular genome 

<img src="files/tig10dotplot.jpeg">


<span style="color:blue;font-size:14pt">We then align the contig agains itself to find the ends of the repeated regions

In [22]:
nucmer --maxmatch --nosimplify tig00000010.fasta tig00000010.fasta
show-coords -lrcTH out.delta


1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "out.ntref" of length 22175
# construct suffix tree for sequence of length 22175
# (maximum reference length is 536870908)
# (maximum query length is 4294967295)
# CONSTRUCTIONTIME /usr/bin/mummer out.ntref 0.00
# reading input file "/home/ubuntu/git-presentations/shark_mito/tig00000010.fasta" of length 22174
# matching query-file "/home/ubuntu/git-presentations/shark_mito/tig00000010.fasta"
# against subject-file "out.ntref"
# COMPLETETIME /usr/bin/mummer out.ntref 0.01
# SPACE /usr/bin/mummer out.ntref 0.04
4: FINISHING DATA
1	22174	1	22174	22174	22174	100.00	22174	22174	100.00	100.00	tig00000010	tig00000010
1	5753	16477	22174	5753	5698	94.58	22174	22174	25.94	25.70	tig00000010	tig00000010
16477	22174	1	5753	5698	5753	94.58	22174	22174	25.70	25.94	tig00000010	tig00000010


<span style="color:blue;font-size:14pt">This shows that region 1-5753 is 94.58% similar to region 16477-22174. We use samtools to delete region 16477-22174

In [13]:
samtools faidx tig00000010.fasta tig00000010:1-16476 > tig00000010_cut.fasta

<span style="color:blue;font-size:14pt">The dotplot show no repeated regions
<img src="files/tig10_cut.jpeg">

<span style="color:blue;font-size:14pt">Aligning agains the Carcharhinus obscurus mitocondrion genome shows that tig00000010 is shifted. By convention, mitochondrial genomes are reported as lineal genomes starting on the tRNA-phe gene. Canu has no way of finding this gene, so the starting point is mostly random. 


<img src="files/tig10_vs_obscurus.jpeg">

<span style="color:blue;font-size:14pt">We aling Carcharhinus obscurus mitochondrion genome to tig00000010 to find the breaking point. In this case, it is between positions 8309 and 8310

In [15]:
nucmer -p cut_vs_carcharhinus tig00000010_cut.fasta Carcharhinus_obscurus_mitochondrion.fasta
show-coords -lrcTH cut_vs_carcharhinus.delta

1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "cut_vs_carcharhinus.ntref" of length 16477
# construct suffix tree for sequence of length 16477
# (maximum reference length is 536870908)
# (maximum query length is 4294967295)
# CONSTRUCTIONTIME /usr/bin/mummer cut_vs_carcharhinus.ntref 0.00
# reading input file "/home/ubuntu/git-presentations/shark_mito/Carcharhinus_obscurus_mitochondrion.fasta" of length 16706
# matching query-file "/home/ubuntu/git-presentations/shark_mito/Carcharhinus_obscurus_mitochondrion.fasta"
# against subject-file "cut_vs_carcharhinus.ntref"
# COMPLETETIME /usr/bin/mummer cut_vs_carcharhinus.ntref 0.01
# SPACE /usr/bin/mummer cut_vs_carcharhinus.ntref 0.03
4: FINISHING DATA
1	8309	8239	16706	8309	8468	91.96	16476	16706	50.43	50.69	tig00000010:1-16476	KC470543.1
8310	16476	1	8238	8167	8238	95.29	16476	16706	49.57	49.31	tig00000010:1-16476	KC470543.1


<span style="color:blue;font-size:14pt">We rotate the genome using samtools

In [24]:
samtools faidx tig00000010_cut.fasta 'tig00000010:1-16476':8310-16476 'tig00000010:1-16476':1-8309 | grep -v '>' | tr -d '\n' | sed '1s/^/>tig00000010_cut_rotated\n/' | sed -e '$a\'  > tig00000010_cut_rotated.fasta

<span style="color:blue;font-size:14pt">The dotplot show tig00000010 has the standar starting point now
<img src="files/cut_rotated_vs_obscurus.jpeg">

<span style="color:blue;font-size:14pt"> We use <a href="http://mitofish.aori.u-tokyo.ac.jp/annotation/input.html">MitoFish server</a> to anotate tig00000010. This gives us a <a href="files/tig00000010_cut_rotated_genes.fa">fasta file</a> with genes. We then blast them agains nt. 
 

In [31]:
blastn -db nt -query tig00000010_cut_rotated_genes.fa -out genes.blast -outfmt "6 qseqid sseqid pident length evalue sscinames salltitles sallacc sstart send qstart qend" -max_target_seqs 1 -num_threads 16 -culling_limit 1 -max_hsps 1 -evalue 0.001

<span style="color:blue;font-size:14pt">We count how many genes blast assings to each shark species

In [32]:
cut -f6 genes.blast | sort | uniq -c | sort -nr

     22 Carcharhinus falciformis
      9 Carcharhinus obscurus
      1 Sphyrna lewini
      1 Prionace glauca
      1 Mustelus widodoi
      1 Carcharhinus macloti
      1 Carcharhinus brevipinna
      1 Carcharhinus amblyrhynchos


<span style="color:blue;font-size:14pt">With this information we can conclude that the sample was Carcharhinus falciformis